# 数据处理与应用

**常用数据统计分析建模方法**
- 描述性统计量和统计图
- 参数估计
- 假设检验
- 回归分析
- 聚类分析

## 描述性统计量与统计图
- 描述性统计量
  - 均值、方差、标准差、最大值、最小值、极差、中位数、分位数、众数、变异系数、中心矩、原点矩、偏度、峰度、协方差、相关系数
- 统计图
  - 箱线图、直方图、经验分布函数图、正态概率图、P-P图、Q-Q图

首先，导入需要的`python`库，如下所示：

In [1]:
import numpy as np
import pandas as pd
import math

### 练习1
- 按班级分别计算身高、体重、肺活量的均值、标准差、最大最小值等
- 提取身高、体重、肺活量、耐力项分数、柔韧和力量分数、速度和灵巧分数，计算相关系数矩阵
- 统计一个数组中各个数字（元素）出现的频数、频率

具体内容见附件：体测成绩.xls,

In [2]:
dataE1_1 = pd.read_excel('./体测成绩.xls', sheet_name ='Sheet1')
students_1 = pd.DataFrame({'class': dataE1_1['班级'],
                           'height': dataE1_1['身高'],
                           'weight': dataE1_1['体重'],
                           'lung_capacity': dataE1_1['肺活量'],
                           'endurance': dataE1_1['耐力类项目分数'],
                           'flexibility':dataE1_1['柔韧及力量类项目分数'],
                           'speed': dataE1_1['速度及灵巧类项目分数']})
grouped_1 = students_1.groupby('class')
for name, group in grouped_1:
    print('Class:', name)
    print('Height -- Mean:', group['height'].mean(), 
          ' Std:', group['height'].std(), 
          ' Max:', group['height'].max(), 
          ' Min:', group['height'].min())
    print('Weight -- Mean:', group['weight'].mean(), 
          ' Std:', group['weight'].std(), 
          ' Max:', group['weight'].max(), 
          ' Min:', group['weight'].min())
    print('Lung capacity -- Mean:', group['lung_capacity'].mean(), 
          ' Std:', group['lung_capacity'].std(), 
          ' Max:', group['lung_capacity'].max(), 
          ' Min:', group['lung_capacity'].min())
    print()

Class: 90401
Height -- Mean: 169.51052631578946  Std: 7.700136704186357  Max: 181.8  Min: 158.0
Weight -- Mean: 58.73157894736841  Std: 11.719170080190024  Max: 93.5  Min: 44.7
Lung capacity -- Mean: 3769.3684210526317  Std: 1352.975125439666  Max: 7359  Min: 1609

Class: 90402
Height -- Mean: 164.94444444444446  Std: 9.367416758745502  Max: 185.7  Min: 149.2
Weight -- Mean: 55.32222222222222  Std: 6.818793713877718  Max: 68.6  Min: 42.0
Lung capacity -- Mean: 3545.1666666666665  Std: 752.8684167510127  Max: 4921  Min: 2479



In [3]:
selected_cols = ['height', 'weight', 'lung_capacity', 'endurance', 'flexibility', 'speed']
corr_matrix = students_1[selected_cols].corr()
print(corr_matrix)

                 height    weight  lung_capacity  endurance  flexibility   
height         1.000000  0.630507       0.460875  -0.287584    -0.514203  \
weight         0.630507  1.000000       0.613563  -0.176182    -0.507716   
lung_capacity  0.460875  0.613563       1.000000  -0.488593    -0.343174   
endurance     -0.287584 -0.176182      -0.488593   1.000000     0.292965   
flexibility   -0.514203 -0.507716      -0.343174   0.292965     1.000000   
speed         -0.291847 -0.072920      -0.069733   0.309928     0.309234   

                  speed  
height        -0.291847  
weight        -0.072920  
lung_capacity -0.069733  
endurance      0.309928  
flexibility    0.309234  
speed          1.000000  


In [4]:
students_2 = pd.DataFrame({'grades': dataE1_1['身高体重等级']})
tot_student = len(students_2.values[:,0])
grouped_2 = students_2.groupby('grades')
print("Value", "Count", "Percent")
for name, group in grouped_2:
    percent = 1.0 * len(group) / tot_student * 100 
    print(name, len(group), "{0}%".format(round(percent, 2)))

Value Count Percent
正常体重 14 37.84%
肥胖 3 8.11%
营养不良 5 13.51%
超重 1 2.7%
较低体重 14 37.84%


## 参数估计

无论总体 X 的分布函数 $F(x;\theta_1, \theta_2, \cdots, \theta_k)$ 的类型已知或未知，我们总是需要去估计某些未知参数或数字特征，这就是参数估计问题。即参数估计就是从样本$(X_1, X_2, \cdots, X_n)$出发，构造一些统计量$\theta_i$ ，其中$\theta_i(X_1, X_2, \cdots, X_n) (i = 1, 2, \cdots, k)$去估计总体X中的某些参数（或数字特征）$\theta_i(i = 1, 2, \cdots, k)$ 。 这样的统计量称为估计量

一般有两种方法完成想要的参数估计

1. 点估计：构造$(X_1, X_2, \cdots, X_n)$ 的函数 $\theta_i(X_1, X_2, \cdots, X_n) (i = 1, 2, \cdots, k)$ 作为参数$\theta_i$的点估计量，称统计量 $\theta_i$为总体X参数$\theta_i$的点估计量
2. 区间估计：构造两个函数  $\theta_{i1}(X_1, X_2, \cdots, X_n)$和 $\theta_{i2}(X_1, X_2, \cdots, X_n)$
做成区间，把这$(\theta_{i1}, \theta_{i2})$ 作为参数$\theta_i$的区间估计.

点估计的做法有两种，矩估计法和极大似然估计法

首先导入我们需要的库以完成对应的题目要求


In [5]:
import scipy.stats as st
import statsmodels.stats.weightstats as sw

### 练习2.1：
某工厂生产的滚珠中随机抽取10个，测得滚珠的直径如下：

`15.14 14.81 15.11 15.26 15.08 15.17 15.12 14.95 15.05 14.87`

 若直径服从正态分布，求 的最大似然估计和置信水平为90%的置信区间
### 解答
 根据题意，滚珠直径服从正态分布，我们需要对其均值进行最大似然估计和置信区间估计。假设滚珠直径的真实均值为 $\mu$，方差为 $\sigma^2$，那么样本的似然函数为：

$L(\mu, \sigma^2) = \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi\sigma^2}} \exp \left(-\frac{(x_i - \mu)^2}{2\sigma^2}\right)$

对上式取对数，并对 $\sigma^2$ 求偏导，可以得到 $\mu$ 的最大似然估计为样本均值 $\bar{x}$，即：

$\hat{\mu} = \bar{x} = \frac{1}{n} \sum_{i=1}^{n} x_i$

接下来进行置信区间估计，考虑到样本量较小（$n=10$），我们可以使用 t 分布来构造置信区间。由于样本方差不是一个无偏估计，因此我们需要对样本方差进行校正，使用如下公式：

$s^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \bar{x})^2$

那么我们可以使用下式构造 $\mu$ 的 90% 置信区间：

$\bar{x} - t_{n-1,\alpha/2} \frac{s}{\sqrt{n}} \leq \mu \leq \bar{x} + t_{n-1,\alpha/2} \frac{s}{\sqrt{n}}$

其中 $t_{n-1,\alpha/2}$ 表示自由度为 $n-1$ 的 t 分布上分位点为 $\alpha/2$。

In [6]:
# 样本数据
x = np.array([15.14, 14.81, 15.11, 15.26, 15.08, 15.17, 15.12, 14.95, 15.05, 14.87])

# 样本均值
x_mean = np.mean(x)

# 样本方差
s2 = np.sum((x - x_mean) ** 2) / (len(x) - 1)

# 最大似然估计
mu_mle = x_mean

# 置信区间估计
alpha = 0.1
t_value = t.ppf(1 - alpha/2, df=len(x)-1)
mu_ci_low = x_mean - t_value * np.sqrt(s2/len(x))
mu_ci_high = x_mean + t_value * np.sqrt(s2/len(x))

print('最大似然估计的均值为:', mu_mle)
print(f'{1-alpha:.0%}的置信区间为: [{mu_ci_low:.4f}, {mu_ci_high:.4f}]')


NameError: name 't' is not defined

## 假设检验


### 练习3.1 z检验

某切割正常工作时，切割的金属棒长度服从正态分布$N(100,4)$.从该
切割机切割的一批金属棒随机抽取15根，测得长度如下：

`97 102 105 112 99 103 102 94 100 95 105 98 102 100 103`

假设总体方差不变，取显著性水平$\alpha = 0.05$ 。检验该切割机工作是否正常。

- 假设$H_0:\mu = \mu_0 = 100$ 正常  $H_1: \mu \neq \mu_0$ 异常

In [ ]:
dataEx3_1 = np.array([97, 102, 105, 112, 99, 
                      103, 102, 94, 100, 95, 
                      105, 98, 102, 100, 103])

mu0 = 100
sigma = 2
Alpha = 0.05

t, p = sw.ztest(dataEx3_1, value=mu0)
if p > Alpha:
    print('we can acknowledge the prediction:')
else:
    print('we can not acknowledge the prediction:')
    
print('The machine works normally')

we can acknowledge the prediction:
The machine works normally


### 练习 3.2 t检验
化肥厂用自动包装机包装化肥，测得某日9包化肥质量(单位kg)为：

`49.4 50.5 50.7 51.7 49.8 47.9 49.2 51.4 48.9`

设每包化肥质量服从正态分布，是否可以认为每包化肥平均质量为50kg.
- 假设$H_0:\mu = \mu_0 = 50$ 正常  $H_1: \mu \neq \mu_0$ 异常

In [ ]:
dataEx3_2 = np.array([49.4, 50.5, 50.7,
                      51.7, 49.8, 47.9, 
                      49.2, 51.4, 48.9])
mu0 = 50
t, p = st.ttest_1samp(dataEx3_2, mu0)
if p > 0.05:
    print('we can acknowledge the prediction:')
else:
    print('we can not acknowledge the prediction:')
    

we can acknowledge the prediction:


其余检验利用matlab实现如下:

```matlab
%% ex6.3-3 总体均值未知时的单个正态总体方差的检验
x = [49.4  50.5  50.7  51.7  49.8  47.9  49.2  51.4  48.9];
var0 = 1.5;
alpha = 0.05;
tail = 'both';
[h,p,varci,stats] = vartest(x,var0,alpha,tail)

%% ex6.3-4 总体标准差未知时的两个正态总体均值的比较检验（独立样本）
x = [20.1,  20.0,  19.3,  20.6,  20.2,  19.9,  20.0,  19.9,  19.1,  19.9];
y = [18.6,  19.1,  20.0,  20.0,  20.0,  19.7,  19.9,  19.6,  20.2];
alpha = 0.05;
tail = 'both';
vartype = 'equal';
[h,p,muci,stats] = ttest2(x,y,alpha,tail,vartype)

%% ex6.3-4x 总体标准差未知时的两个正态总体均值的比较检验（配对样本）
% x = [80.3,68.6,72.2,71.5,72.3,70.1,74.6,73.0,58.7,78.6,85.6,78.0];
% y = [74.0,71.2,66.3,65.3,66.0,61.6,68.8,72.6,65.7,72.6,77.1,71.5];
% Alpha = 0.05;
% tail = 'both';
% [h,p,muci,stats] = ttest(x,y,Alpha,tail)

%% ex6.3-5 总体均值未知时的两个正态总体方差的比较检验
x = [20.1,  20.0,  19.3,  20.6,  20.2,  19.9,  20.0,  19.9,  19.1,  19.9];
y = [18.6,  19.1,  20.0,  20.0,  20.0,  19.7,  19.9,  19.6,  20.2];
alpha = 0.05;
tail = 'both';
[h,p,varci,stats] = vartest2(x,y,alpha,tail)
```